In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('./data/train.csv', encoding='windows-1251')

/home/fisher/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,Interval,Date,OrderDate,ClientID,ChannelID,OrderID,MaterialID,GroupID,Cluster,CancelFlag,OrderCnt,DeliveryType,prepay,count_edit
0,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3328810.0,61.0,NaN,0,1.0,Обычная доставка,0,1
1,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3281258.0,30.0,NaN,0,2.0,Обычная доставка,0,1
2,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3210734.0,10.0,NaN,0,1.0,Обычная доставка,0,1
3,14-16.,03/10/2018,02/10/2018,93808186,2,90102063002,3328848.0,61.0,NaN,0,2.0,Обычная доставка,0,1
4,12-14.,07/10/2018,05/10/2018,94112406,2,90102091007,3347801.0,17.0,NaN,0,10.0,Обычная доставка,0,1


In [12]:
features = ['GroupID', 'MaterialID']
important_features = []

for feature in features:
    t = df[feature].value_counts()
    t0 = df[df['CancelFlag'] == 0][feature].value_counts()
    t1 = df[df['CancelFlag'] == 1][feature].value_counts()
    t0_ = t0[t0>100]
    t1_ = t1[t1>100]
    t0_part = t0_ / t0.sum()
    t1_part = t1_ / t1.sum()
    
    diff = t1_part / t0_part
    sorted_diff = diff.apply(lambda x: 1 / x if x < 1 else x).sort_values(ascending=False)
    for key in sorted_diff[sorted_diff > 1.4].keys():
        important_features.append(f'{feature}_{key}')

In [17]:
feature = 'Interval'
t = df[feature].value_counts()
t0 = df[df['CancelFlag'] == 0][feature].value_counts()
t1 = df[df['CancelFlag'] == 1][feature].value_counts()
t0_ = t0[t0>100]
t1_ = t1[t1>100]
t0_part = t0_ / t0.sum()
t1_part = t1_ / t1.sum()

diff = t1_part / t0_part
sorted_diff = diff.apply(lambda x: 1 / x if x < 1 else x).sort_values(ascending=False)
sorted_diff[sorted_diff > 1.4]

10-18.    35.506374
14-17.     2.280324
22-2.      2.066545
0-2.       1.886888
8-13.      1.819420
13-18.     1.803632
21-23.     1.761921
20-21.     1.550835
6-14.      1.538576
19-20.     1.494193
22-23.     1.477141
22-0.      1.403678
Name: Interval, dtype: float64

In [21]:
df.Interval.value_counts() / 5041

10-12.    251.784765
16-18.    235.470145
8-10.     224.915890
18-20.    201.998016
20-22.    193.155525
12-14.    165.120413
14-16.    149.560801
6-8.       95.671295
22-0.      80.713549
10-14.     44.698869
8-13.      25.580837
22-2.      24.714342
19-21.     21.125372
10-16.     18.565364
6-14.      14.862527
21-23.     12.216029
13-18.     10.770680
0-2.        6.079151
15-17.      4.156913
19-23.      2.280897
19-20.      1.786154
20-21.      1.635390
21-22.      1.217417
22-23.      1.182107
14-17.      0.337036
8-14.       0.120611
10-18.      0.115652
9-16.       0.051577
16-22.      0.047808
8-18.       0.024003
Name: Interval, dtype: float64

In [20]:
df[df['CancelFlag'] ==1]['Interval'].value_counts()

10-12.    63346
16-18.    58429
8-10.     56935
20-22.    55537
18-20.    52686
14-16.    46670
12-14.    41773
22-0.     31204
6-8.      27588
22-2.     13575
10-14.    12790
8-13.     12534
6-14.      6252
10-16.     6010
13-18.     5236
19-21.     4349
0-2.       3078
21-23.     2001
15-17.     1083
19-23.      852
22-23.      479
10-18.      395
19-20.      343
20-21.      303
21-22.      290
14-17.      202
9-16.        69
16-22.       51
8-18.        34
8-14.        16
Name: Interval, dtype: int64

In [13]:
important_features

['GroupID_12.0',
 'GroupID_53.0',
 'GroupID_2.0',
 'MaterialID_3080210.0',
 'MaterialID_3038501.0',
 'MaterialID_3342911.0',
 'MaterialID_3181726.0',
 'MaterialID_3221215.0',
 'MaterialID_3271717.0',
 'MaterialID_3291366.0',
 'MaterialID_3286521.0',
 'MaterialID_3258660.0',
 'MaterialID_3262803.0',
 'MaterialID_3225582.0',
 'MaterialID_3203153.0',
 'MaterialID_3089945.0',
 'MaterialID_3075886.0',
 'MaterialID_3312483.0',
 'MaterialID_3057685.0',
 'MaterialID_3168358.0',
 'MaterialID_3324388.0',
 'MaterialID_3312195.0',
 'MaterialID_3129509.0',
 'MaterialID_3062473.0',
 'MaterialID_3344549.0',
 'MaterialID_3316697.0',
 'MaterialID_3152401.0',
 'MaterialID_3274594.0',
 'MaterialID_3342912.0',
 'MaterialID_2013427.0',
 'MaterialID_3345817.0',
 'MaterialID_2011369.0',
 'MaterialID_3252055.0',
 'MaterialID_3286552.0',
 'MaterialID_3286568.0',
 'MaterialID_3071758.0',
 'MaterialID_2013496.0',
 'MaterialID_3077019.0',
 'MaterialID_3057966.0',
 'MaterialID_2012027.0',
 'MaterialID_3078750.0',


In [6]:
df["Cluster"] = df["Cluster"].fillna('999')
#groups_counts = list(df['GroupID'].value_counts()[df['GroupID'].value_counts() > 10000].keys())
#df['GroupID'] = df['GroupID'].apply(lambda x: x if x in groups_counts else 999)
#materials_counts = list(df['MaterialID'].value_counts()[df['MaterialID'].value_counts() > 10000].keys())
#df['MaterialID'] = df['MaterialID'].apply(lambda x: x if x in materials_counts else 999)

In [11]:
df.MaterialID.value_counts()

2013197.0    60809
3186418.0    40935
2013264.0    37670
2013421.0    32945
2012529.0    32239
             ...  
3316250.0        1
3325424.0        1
2903364.0        1
2903376.0        1
2902413.0        1
Name: MaterialID, Length: 51082, dtype: int64

In [7]:
cols = ['OrderID', 'Interval', 'Date', 'OrderDate', 'ClientID', 'ChannelID','CancelFlag',
       'Cluster', 'DeliveryType', 'prepay', 'count_edit']

In [8]:
df_ = df.groupby(cols).sum().reset_index()[['OrderID', 'Interval', 'Date', 'OrderDate', 'ClientID', 'ChannelID','CancelFlag',
       'Cluster', 'DeliveryType', 'prepay', 'count_edit', 'OrderCnt']]

In [9]:
orders = df_['OrderID']

In [10]:
df_dummies = pd.get_dummies(df[['OrderID', 'MaterialID', 'GroupID']], columns=['MaterialID', 'GroupID'])

MemoryError: Unable to allocate array with shape (9023184, 51082) and data type uint8

In [15]:
df_group1 = df_dummies.head(5000000).groupby('OrderID').sum()
df_group2 = df_dummies.tail(len(df_dummies) - 5000000 - 1).groupby('OrderID').sum()

In [25]:
df_group = df_group1.append(df_group2)

In [30]:
df_ = df_.merge(df_group, on='OrderID', right_index=True)

In [32]:
df_['OrderDate_weekday'] = df_['OrderDate'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').weekday())
df_['Date_weekday'] = df_['Date'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').weekday())

In [33]:
def to_labels(series):
    le = LabelEncoder()
    return le.fit_transform(series)

In [38]:
cols = ['Interval', 'ChannelID', 'DeliveryType', 'Cluster']

for col in cols:
    df_[col] = to_labels(df_[col])

In [39]:
df_

,OrderID,Interval,Date,OrderDate,ClientID,ChannelID,CancelFlag,Cluster,DeliveryType,prepay,...,GroupID_41.0,GroupID_42.0,GroupID_46.0,GroupID_55.0,GroupID_59.0,GroupID_61.0,GroupID_63.0,GroupID_66.0,OrderDate_weekday,Date_weekday
0,90102063002,7,03/10/2018,02/10/2018,93808186,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1,2
1,90102091007,5,07/10/2018,05/10/2018,94112406,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,6
2,90102092000,14,05/10/2018,05/10/2018,93696397,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
3,90102103017,20,12/10/2018,12/10/2018,93696397,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,4
4,90102104012,4,17/10/2018,15/10/2018,93411902,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354846,98357278690,7,31/12/2018,30/12/2018,93324410,0,0,18,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,6,0
354847,98357278797,20,30/12/2018,30/12/2018,94273838,4,0,18,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6
354848,98357279051,7,31/12/2018,30/12/2018,93899170,0,0,18,1,0,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0
354849,98357281094,25,31/12/2018,30/12/2018,91690087,0,0,18,1,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,6,0


In [40]:
df_res = df_.drop(['OrderID', 'Date', 'OrderDate', 'ClientID'], axis=1)

In [42]:
df_res.to_csv('./data/data_pre.csv', index=False)